# MUBen: Usage Demonstration

In this demonstration, we'll guide you through a foundational training and testing of MUBen using the BBBP dataset as a basic example.
We've chosen the DNN as our backbone model because it is both efficient and offers satisfactory performance.
For uncertainty quantification (UQ), we'll evaluate both the Deterministic method (referred to as "none" within MUBen) and Temperature Scaling.
While the procedures for other backbone models, UQ methods, or datasets are largely similar, you can explore specific variations by referring to the scripts in the `<project root>/run/` directory.

In [1]:
# Import the necessary packages
import sys
# add the root path of the project to the python path
sys.path.append('../')
import logging
import wandb
from transformers import set_seed

# from muben.dnn.dataset import Dataset
# from muben.dnn.args import Config
# from muben.dnn.train import Trainer
from muben.utils.selectors import argument_selector, configure_selector, dataset_selector, model_selector
from muben.args import DescriptorArguments
from muben.train import Trainer

from muben.utils.io import set_logging

# initialize logger
logger = logging.getLogger(__name__)

## Run the Deterministic UQ method

In [2]:
# Set up the logging format and random seed.
# We do not use wandb for this demo, so we set its mode to "disabled".
set_logging()
set_seed(42)
wandb.init(mode="disabled",)

# Select the classes based on the descriptor type.
descriptor_type = "RDKit"
config_class = configure_selector(descriptor_type)
dataset_class = dataset_selector(descriptor_type)
model_class = model_selector(descriptor_type)

# Specify the configuration of the experiment.
# Notice that although we directly edit the config object here, a more appropriate way of doing this is 
# passing arguments through the shell or json scripts when we are running the experiments through the terminal.
config = config_class()
config.model_name = "DNN"
config.feature_type = "rdkit"
config.data_folder = "../data/files/"
config.dataset_name = "bbbp"
config.result_folder = "../output-demo/"
config.uncertainty_method = "none"
config.retrain_model = True

# We only train the model for a few epochs for the demo.
config.n_epochs = 50
# activate training timer
config.time_training = True

# Post initialization of the arguments.
config.__post_init__()

# Load dataset metadata, validate the arguments, and log the configuration.
_ = config.get_meta().validate().log()


02/28/2024 19:03:47 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
02/28/2024 19:03:47 - INFO - muben.args.args -   Configurations:
{
  "activation": "ReLU",
  "al_random_sampling": false,
  "apply_preconditioned_sgld": false,
  "apply_temperature_scaling_after_focal_loss": false,
  "apply_wandb": null,
  "batch_size": 32,
  "batch_size_inference": null,
  "bbp_prior_sigma": 0.1,
  "binary_classification_with_softmax": false,
  "classes": [0, 1],
  "d_dnn_hidden": 128,
  "d_feature": 200,
  "data_dir": "../data/files/bbbp",
  "data_folder": "../data/files/",
  "data_seed": null,
  "dataset_name": "bbbp",
  "debug": false,
  "deploy": false,
  "descriptor_type": null,
  "device": "cuda",
  "disable_dataset_saving": false,
  "disable_result_saving": false,
  "disable_wandb": false,
  "dropout": 0.1,
  "enable_active_learning": false,
  "eval_metric": null,
  "evidenti

In [3]:
# Load and process the training, validation and test datasets
training_dataset = Dataset().prepare(config=config, partition="train")
valid_dataset = Dataset().prepare(config=config, partition="valid")
test_dataset = Dataset().prepare(config=config, partition="test")

09/11/2023 13:18:53 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/train.pt
09/11/2023 13:18:53 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/valid.pt
09/11/2023 13:18:53 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/test.pt


In [4]:
# Inintialized the trainer with the configuration and datasets
trainer = Trainer(
    config=config,
    training_dataset=training_dataset,
    valid_dataset=valid_dataset,
    test_dataset=test_dataset,
)

09/11/2023 13:18:53 - INFO - muben.base.train.trainer -   Trainer initialized. The model contains 157953 parameters


In [5]:
# Run the training, validation and test process.
# The model checkpoint and predicted results will be automatically saved in the specified output folder.
trainer.run()

09/11/2023 13:18:54 - INFO - muben.base.train.trainer -   Training model
09/11/2023 13:18:54 - INFO - muben.base.train.trainer -   [Training Epoch 0]
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     Training Loss: 0.5515.
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     Average Time per Step: 11.3329.
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -   [Valid step 1] results:
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     roc_auc: 0.3509.
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -   Model buffer is updated!
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -   [Training Epoch 1]
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     Training Loss: 0.4296.
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     Average Time per Step: 2.0877.
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -   [Valid step 2] results:
09/11/2023 13:18:55 - INFO - muben.base.train.trainer -     roc_auc: 0.8983.
09/11/2023 13:18:55 - I

## Run Temperature Scaling

In [6]:
wandb.init(mode="disabled",)
# Change some configuration items.
config.uncertainty_method = "TemperatureScaling"
config.retrain_model = False
config.n_ts_epochs = 10  # number of epochs for training the temperature scaling layer.
config.__post_init__()
_ = config.validate().log()

09/11/2023 13:19:01 - INFO - muben.base.args -   Configurations:
{
  "apply_preconditioned_sgld": false,
  "apply_temperature_scaling_after_focal_loss": false,
  "apply_wandb": true,
  "batch_size": 32,
  "batch_size_inference": null,
  "bbp_prior_sigma": 0.1,
  "binary_classification_with_softmax": false,
  "classes": [0, 1],
  "d_dnn_hidden": 128,
  "d_feature": 200,
  "data_dir": "../data/files/bbbp",
  "data_folder": "../data/files/",
  "data_seed": null,
  "dataset_name": "bbbp",
  "debug": false,
  "deploy": false,
  "device": "cuda",
  "disable_dataset_saving": false,
  "disable_result_saving": false,
  "disable_wandb": false,
  "dropout": 0.1,
  "eval_metric": null,
  "evidential_clx_loss_annealing_epochs": 10,
  "evidential_reg_loss_weight": 1,
  "feature_type": "rdkit",
  "freeze_backbone": false,
  "grad_norm": 0,
  "ignore_no_uncertainty_output": false,
  "ignore_preprocessed_dataset": false,
  "ignore_uncertainty_output": false,
  "k_swa_checkpoints": 20,
  "log_path": nul

In [7]:
# Re-inintialized the trainer with the updated configuration.
# The datasets are not changed.
trainer = Trainer(
    config=config,
    training_dataset=training_dataset,
    valid_dataset=valid_dataset,
    test_dataset=test_dataset,
)

09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   Trainer initialized. The model contains 157953 parameters


In [8]:
# Run the training, validation and test process.
# The trainer will load the model checkpoint from the Deterministic run and
# continue training the temperature scaling layer.
# Notice that not all UQ methods support continued training. For example, BBP requires training from scratch.
trainer.run()

09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   Loading trained model from ../output-demo/bbbp/DNN-rdkit/TemperatureScaling/seed-0/model_best.ckpt.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   Temperature Scaling session start.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   Training model on validation
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   [Training Epoch 0]
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -     Training Loss: 0.4966.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -     Average Time per Step: 1.5625.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   [Training Epoch 1]
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -     Training Loss: 0.4961.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -     Average Time per Step: 1.3252.
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -   [Training Epoch 2]
09/11/2023 13:19:01 - INFO - muben.base.train.trainer -     Training Loss: 0.4961.


## Calculating Metrics
This is a simplified version of metric calculation. Please check `<project root>/assist/result_get_metrics.py` for the full function.

In [9]:
import os.path as op
import pandas as pd
from muben.utils.metrics import classification_metrics
from muben.utils.io import load_results

09/11/2023 13:19:01 - INFO - torch.distributed.nn.jit.instantiator -   Created a temporary directory at /tmp/tmp82lgbnu7
09/11/2023 13:19:01 - INFO - torch.distributed.nn.jit.instantiator -   Writing /tmp/tmp82lgbnu7/_remote_module_non_scriptable.py


In [10]:
# Define the path to the predicted results. "det" stands for "Deterministic"; "ts" stands for "Temperature Scaling".
det_result = op.join(
    config.result_folder, config.dataset_name, f"{config.model_name}-{config.feature_type}",
    "none", f"seed-{config.seed}", "preds", "0.pt"
)
ts_result = op.join(
    config.result_folder, config.dataset_name, f"{config.model_name}-{config.feature_type}",
    "TemperatureScaling", f"seed-{config.seed}", "preds", "0.pt"
)

# Load the predicted results.
det_preds, _, lbs, masks = load_results([det_result])
ts_preds, _, _, _ = load_results([ts_result])

In [11]:
# Calculate the metrics.
det_metrics = classification_metrics(det_preds, lbs, masks)
ts_metrics = classification_metrics(ts_preds, lbs, masks)

det_metrics = {k: v['macro-avg'] for k, v in det_metrics.items()}
ts_metrics = {k: v['macro-avg'] for k, v in ts_metrics.items()}

In [12]:
# Present the results in a dataframe.
det_metrics_df = pd.DataFrame({"Deterministic": det_metrics, "TemperatureScaling": ts_metrics})
det_metrics_df.T

,roc-auc,prc-auc,ece,mce,nll,brier
Deterministic,0.697434,0.749759,0.363339,0.634762,1.497666,0.352379
TemperatureScaling,0.682195,0.733970,0.217886,0.546396,0.877666,0.281391
